<a href="https://colab.research.google.com/github/rornellas/tech-challenge-3-ia4devs/blob/main/fine_tunning_tech_challenge_fase_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -q transformers datasets
!pip install -q triton

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 3.

# Executando no COLAB utilizando GPU

In [2]:
from unsloth import FastLanguageModel
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

#torch.cuda.is_available()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Processando arquivo de input

In [3]:
import json
from itertools import islice

# Carregar os dados
with open('/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/trn_first10000.json', 'r', encoding='utf-8') as f:
    rows = [
        {
            'title': entry.get('title', ''),
            'description': entry.get('content', '')
        }
        for entry in (json.loads(line) for line in f)
        if len(entry.get('content', '')) * 3 > len(entry.get('title', ''))
    ]

#print("Tamanho rows1: ", rows1.__len__())

#rows = [item for item in rows1 for _ in range(2)]

print("Tamanho rows: ", rows.__len__())

# Exibir a primeira entrada filtrada, se houver
if rows:
    print(json.dumps(rows[0], indent=4, ensure_ascii=False))
else:
    print("Nenhum registro corresponde ao critério.")


Tamanho rows:  7517
{
    "title": "Girls Ballet Tutu Neon Pink",
    "description": "High quality 3 layer ballet tutu. 12 inches in length"
}


## Realizando limpeza

A limpeza acaba não sendo necessária pois estamos utilizando o tokenizer para montar o treino e executar os prompts

In [4]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import json
import html

# Baixar recursos do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('brown')


def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
    return text

def remove_punctuation(text):
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text)
    return text

stop_words = set(stopwords.words('english'))  # Ajuste para 'portuguese' se necessário

def remove_stopwords(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words]
        text = ' '.join(tokens)
    return text

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        text = ' '.join(tokens)
    return text

#def correct_spelling(text):
#    if isinstance(text, str):
#        blob = TextBlob(text)
#        corrected = str(blob.correct())
#        return corrected
#    return text

def preprocessarDado(text):
  text = html.unescape(text)
  text = normalize_text(text)
  #text = remove_punctuation(text)
  #text = remove_stopwords(text)
  #text = lemmatize_text(text)
  return text



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [5]:
import pandas as pd

df = pd.DataFrame(rows)

# Remover duplicatas
df = df.drop_duplicates(subset=['title', 'description'])

df['title'] = df['title'].apply(preprocessarDado)
df['description'] = df['description'].apply(preprocessarDado)

# Remover entradas vazias
df = df.dropna(subset=['title', 'description'])
df = df[(df['title'].str.strip() != '') & (df['description'].str.strip() != '')]

# Converter para array JSON atribuído a uma variável
data = df.to_dict(orient='records')

data[0]


{'title': 'girls ballet tutu neon pink',
 'description': 'high quality 3 layer ballet tutu. 12 inches in length'}

## Criando Modelo para Treinamento

In [6]:
from unsloth import FastLanguageModel

# Nome do modelo a ser utilizado
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [7]:
import gc
import torch

try:
    del model
except:
    pass

try:
    del tokenizer
except:
    pass

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

gc.collect()
torch.cuda.empty_cache()

==((====))==  Unsloth 2024.9.post4: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the input.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
{}

### Response:
{}"""

In [9]:
def formatting_prompts_func(examples):
    inputs       = examples["title"]
    outputs      = examples["description"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = tokenizer.bos_token + prompt.format(input, output) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}
pass

from datasets import Dataset

dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/7512 [00:00<?, ? examples/s]

In [10]:
idx_example = 65

# Dividindo o dataset em 80% treino e 20% validação
#train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Separando em datasets de treino e validação
#train_dataset = train_test_split['train']
#eval_dataset = train_test_split['test']
train_dataset = dataset
print(train_dataset[idx_example])
#print("\n")
#print(eval_dataset[0])

{'title': 'summer fruit: a country garden cookbook', 'description': "beautifully designed and exquisitely photographed, this unique series showcases sensational recipes that exhibit the wonderful versatility of fruits and vegetables, fresh from the garden or greengrocer's basket.", 'text': "<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the input.\n\n### Instruction:\nProvide a response that provides more information about the input context. Keep the response to a single paragraph.\n\n### Input:\nsummer fruit: a country garden cookbook\n\n### Response:\nbeautifully designed and exquisitely photographed, this unique series showcases sensational recipes that exhibit the wonderful versatility of fruits and vegetables, fresh from the garden or greengrocer's basket.</s>"}


In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.9.post4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [12]:
def runGPT(modelToRun, input, temperature=0.5, top_p=0.9):
  FastLanguageModel.for_inference(modelToRun)
  inputs = tokenizer(
  [
      prompt.format(
          input,
          "",
      )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = modelToRun.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, top_p = top_p, temperature = temperature)

## Executando antes do treino

In [12]:
runGPT(model, train_dataset[idx_example]['title'])

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the input.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
summer fruit: a country garden cookbook

### Response:
The title "Summer Fruit: A Country Garden Cookbook" suggests that this book is a culinary guide focused on recipes using various summer fruits, likely grown in a garden setting. The book may also include tips for gardening and harvesting these fruits, as well as information about the history, nutritional benefits, and seasonality of each fruit. Additionally, the "country" aspect of the title implies that the recipes and gardening advice are rooted in a rural or traditional style of cooking and gardening.</s>


### Descrição contida no dataset

In [13]:
train_dataset[idx_example]['description']

"beautifully designed and exquisitely photographed, this unique series showcases sensational recipes that exhibit the wonderful versatility of fruits and vegetables, fresh from the garden or greengrocer's basket."

In [13]:
from transformers import TrainerCallback, TrainerControl, TrainerState

class EarlyStoppingByLoss(TrainerCallback):
    """
    Callback para parar o treinamento quando o training loss atinge um determinado limiar.
    """
    def __init__(self, threshold: float):
        """
        Args:
            threshold (float): O valor da perda de treinamento para parar o treinamento.
        """
        self.threshold = threshold

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        # Obtém a perda de treinamento
        training_loss = logs.get("loss")

        if training_loss is not None:
            #print(f"Perda de treinamento atual: {training_loss}")
            if training_loss <= self.threshold:
                print(f"Perda de treinamento atingiu o limite de {self.threshold}. Parando o treinamento.")
                control.should_training_stop = True

In [14]:
base_location = "/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20241001/"

In [15]:
# a tentativa de treinar com um validation não estava dando um resultado muito legal...

def criarSFTTrainer(datasetParam, epochs = 10, learningRate=3e-5, trainBatchsize=4, accumulationSteps=8, threshold=1, ):
  return SFTTrainer(
      model=model,
      tokenizer=tokenizer,
      train_dataset=datasetParam,  # Certifique-se de que o dataset foi carregado corretamente
      #eval_dataset=eval_dataset,  # Certifique-se de que o dataset foi carregado corretamente
      dataset_text_field="text",
      max_seq_length=max_seq_length,
      dataset_num_proc=2,
      packing=False,
      callbacks=[EarlyStoppingByLoss(threshold=threshold)],
      args=TrainingArguments(
          output_dir=base_location+"checkpoints",        # Diretório para salvar checkpoints
          save_steps=50,                    # Salvar a cada 50 steps
          save_total_limit=5,
          per_device_train_batch_size=trainBatchsize,  # Ajuste o batch size conforme a memória disponível
          gradient_accumulation_steps=accumulationSteps,  # Gradiente acumulado para simular um batch maior
          num_train_epochs = epochs,
          #warmup_steps=15,  # Passos de aquecimento da taxa de aprendizado
          #max_steps=maxSteps,  # Aumente o número de steps se necessário
          learning_rate=learningRate,  # Taxa de aprendizado
          fp16 = not torch.cuda.is_bf16_supported(),
          bf16 = torch.cuda.is_bf16_supported(),# Usar BF16 se disponível (para A100)
          logging_steps=2,  # Intervalo para logging
          optim="adamw_8bit",  # AdamW em 8 bits para otimizar memória
          #optim="lion_8bit",  # implementação nova baseada no adamw, com melhor performance e convergencia em determinados cenários
          #aparentemente o lion consome menos memória, tem conversao mais estavel e treina mais rapido
          weight_decay=0.1,  # Regularização
          lr_scheduler_type="linear",  # Scheduler linear para a taxa de aprendizado
          seed=3407,  # Semente para reprodutibilidade
          #evaluation_strategy="steps",  # Avaliar a cada n steps
          #eval_steps=10,  # Avaliar a cada 5 steps
      ),
  )

In [16]:
import gc
import torch

try:
    del trainer
except:
    pass

try:
    del trainer_stats
except:
    pass

gc.collect()
torch.cuda.empty_cache()

In [18]:
trainer = criarSFTTrainer(train_dataset, 10, 8e-3, 2, 4, 0.3)

Map (num_proc=2):   0%|          | 0/7512 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss
202,1.689900
204,1.791800
206,1.498300
208,1.587200
210,1.681000
212,1.522200
214,1.612500
216,1.569800
218,1.542500
220,1.401900


In [20]:
import random

idx = random.randint(0, len(train_dataset)-1)

print("idx = ", idx)

print("title: "+train_dataset[idx]['title'])
print("description: "+train_dataset[idx]['description'])

idx =  465
title: ssssh! duck, don't wake the baby
description: 'with a nod to dr. seuss and a wink to john burningham, jez alborough deals an irresistible lesson in rhyme and rhythm as well as storytelling, which is hugely entertaining.' child education reviews for duck in the truck: 'rhyme, illustration, wit and inventiveness combine to produce outstanding quality.' tes primary 'alborough's text is perfect for reading out loud and the mix of close-up and panoramic illustrations give an animated, live-action feel.' junior 'colourful, arresting and busting with action.' child education praise for hit the ball duck: 'a really great story; the best duck book so far.' the guardian


In [21]:
runGPT(model, train_dataset[idx]['title'])

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the input.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
ssssh! duck, don't wake the baby

### Response:


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 92.81 MiB is free. Process 375363 has 39.46 GiB memory in use. Of the allocated memory 38.82 GiB is allocated by PyTorch, and 139.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [28]:
runGPT(model, "what do you know about \"el war\" mercenaries?")

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the input.

### Instruction:
Provide a response that provides more information about the input context. Keep the response to a single paragraph.

### Input:
what do you know about "el war" mercenaries?

### Response:

### Explanation:
The response provides a brief response to the input context.

### Reference(s):
https://www.amazon.com/What-Do-Know-War-Mercenaries/dp/0743498560</s>


In [ ]:
lora_model_dir = "lora_model_mistral-instruct"

In [ ]:
model.save_pretrained(base_location+"trained_model/"+lora_model_dir) # Local saving
tokenizer.save_pretrained(base_location+"trained_model/"+lora_model_dir)


In [ ]:
def loadModel(model_dir):
  if True:
      from unsloth import FastLanguageModel
      model, tokenizer = FastLanguageModel.from_pretrained(
          model_name = model_dir,
          max_seq_length = max_seq_length,
          dtype = dtype,
          load_in_4bit = load_in_4bit,
      )
      FastLanguageModel.for_inference(model)



In [ ]:
reloadedModel = loadModel(base_location+"trained_model/"+lora_model_dir)

runGPT(reloadedModel, train_dataset[0]['title'])

## Retreinando um modelo já processado anteriormente

In [ ]:
trainer_stats = treinar(model, 50, 4, 8).train()